# Slow Motion with Super SloMo

This notebook uses [Super SloMo](https://arxiv.org/abs/1712.00080) from the open source project [avinashpaliwal/Super-SloMo](https://github.com/avinashpaliwal/Super-SloMo) to slow down a given youtube video or any of your video (modified).


In [ ]:
#@title STEP1: Setup Super SloMo

#@markdown * Installs avinashpaliwal/Super-SloMo
#@markdown * Download pre-trained Model


import os
from os.path import exists, join, basename, splitext, dirname
!pip install gdown

git_repo_url = 'https://github.com/avinashpaliwal/Super-SloMo.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone -q --depth 1 {git_repo_url}
  !pip install -q youtube-dl
  ffmpeg_path = !which ffmpeg
  ffmpeg_path = dirname(ffmpeg_path[0])
  
import sys
sys.path.append(project_name)
from IPython.display import YouTubeVideo


# def download_from_google_drive(file_id, file_name):
#   # download a file from the Google Drive link
#   !rm -f ./cookie
#   !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id={file_id}" > /dev/null
#   confirm_text = !awk '/download/ {print $NF}' ./cookie
#   confirm_text = confirm_text[0]
#   !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm={confirm_text}&id={file_id}" -o {file_name}
  
#download the model from google drive
def download_from_google_drive(file_ids, file_name):
  for id in file_ids:
    print("Attept download from", id)
    response = !gdown --id $id --output $file_name
    if response[0] == 'Downloading...':
      break
    else:
      continue

file_ids = ['1IvobLDbRiBgZr3ryCRrWL8xDbMZ-KnpF', 
            '1xGtOMha86xXdT-MXMroo40SZdAhN6QuS', 
            '1zkDN37dTNLZ7mS6E59XVFaugNI2AUUrJ', 
            '1CVbwZvDDO1ofYFtaIWS9PXjl0ZBnsZM3', 
            '1cVPeLxCZwf0P9dsSA71XkCxZKeGcu_dx']

pretrained_model = 'SuperSloMo.ckpt'

if not exists(pretrained_model):
  download_from_google_drive(file_ids, pretrained_model)



def show_local_mp4_video(file_name, slowmoFactor, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''
                        <script>
                          var vid = document.getElementById("myVideo");
                          vid.playbackRate = 1/{3};
                        </script> 
                        <video width="{0}" height="{1}" alt="test" id="myVideo" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii'), slowmoFactor))

In [ ]:
#@title STEP2A: Super SloMo on a Youtube Video
YOUTUBE_URL = 'https://www.youtube.com/watch?v=QJO3ROT-A4E' #@param {type:"string"}

#find youtube video id
from urllib import parse as urlparse
url_data = urlparse.urlparse(YOUTUBE_URL)
query = urlparse.parse_qs(url_data.query)
YOUTUBE_ID = query["v"][0]

#displays your youtube video
# YouTubeVideo(YOUTUBE_ID)

SLOW_MOTION_FACTOR =  8#@param {type:"integer"}
# TARGET_FPS = 240 #@param {type:"integer"}
#@markdown ---
#@markdown <small>Making slow mo of entire video can take long time so ..</small>

#@markdown ### Trim the video (start, end) seconds
start = 51 #@param {type:"integer"}

end =  52#@param {type:"integer"}


#@markdown ---

interval = end - start

!rm -df youtube.mp4
# download the youtube with the given ID
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

#find the frame_rate of driving video
from fractions import Fraction
frame_rate_source = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate youtube.mp4
frame_rate_source = int(Fraction(frame_rate_source[0]))
print("FPS=", frame_rate_source)

TARGET_FPS = frame_rate_source * SLOW_MOTION_FACTOR

#delete video.mp4 if already exits
!rm -f video.mp4

# cut the video
# !ffmpeg -y -loglevel info -i youtube.mp4 -t 5 video.mp4
!ffmpeg -y -i youtube.mp4 -ss {start} -t {interval} -async 1 video.mp4

#delete if file already exists
!rm -f output.mp4
!rm -f output.mkv

#generate slowmo
!cd {project_name} && python video_to_slomo.py --ffmpeg {ffmpeg_path} --checkpoint ../{pretrained_model} --video ../video.mp4 --sf {SLOW_MOTION_FACTOR} --fps {TARGET_FPS} --output ../output.mkv && ffmpeg -i ../output.mkv ../output.mp4

from IPython.display import clear_output


#find the frame_rate of source video
from fractions import Fraction
frame_rate_source = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate video.mp4
frame_rate_source = float(Fraction(frame_rate_source[0]))

#if fps > 30 overwrite frame_rate_source = 30
if frame_rate_source > 30:
  frame_rate_source = 30

#slow down both video
!ffmpeg  -r {frame_rate_source/SLOW_MOTION_FACTOR} -i video.mp4 -y video_slow.mp4
!ffmpeg -r {frame_rate_source} -i output.mp4 -y output_slow.mp4

#combile two videos
!ffmpeg \
  -i output_slow.mp4 \
  -i video_slow.mp4 \
  -filter_complex '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]' \
  -map [vid] \
  -c:v libx264 \
  -crf 23 \
  -preset veryfast \
  -y output_4_display.mp4
clear_output()
show_local_mp4_video('output_4_display.mp4', width=900, height=253, slowmoFactor = 1)




In [ ]:
#@title STEP2B: Super SloMo on a Your Video
#@markdown <small>Upload your video first then copy its path</small>
PATH_TO_YOU_VIDEO = '/content/clip_try.mp4' #@param {type:"string"}

def path_leaf(path):
  import ntpath
  head, tail = ntpath.split(path)
  return tail or ntpath.basename(head)

VIDEONAME = path_leaf(PATH_TO_YOU_VIDEO)

VIDEONAME = '\"' + VIDEONAME +'\"'


SLOW_MOTION_FACTOR =  8#@param {type:"integer"}
# TARGET_FPS = 240 #@param {type:"integer"}
#@markdown ---
#@markdown <small>Making slow mo of entire video can take long time so ..</small>

#@markdown ### Trim the video (start, end) seconds
start =  0#@param {type:"integer"}

end =  3#@param {type:"integer"}


#@markdown ---

interval = end - start


#find the frame_rate of driving video
from fractions import Fraction
frame_rate_source = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate $PATH_TO_YOU_VIDEO
frame_rate_source = int(Fraction(frame_rate_source[0]))
print("FPS=", frame_rate_source)

TARGET_FPS = frame_rate_source * SLOW_MOTION_FACTOR

#delete if file already exists
!rm -f myCutvideo.mp4
# cut the video
# !ffmpeg -y -loglevel info -i youtube.mp4 -t 5 video.mp4
!ffmpeg -i {VIDEONAME} -ss {start} -t {interval} -async 1 myCutvideo.mp4

#delete if file already exists
!rm -f myoutput.mp4
!rm -f myoutput.mkv

#generate slowmo
!cd {project_name} && python video_to_slomo.py --ffmpeg {ffmpeg_path} --checkpoint ../{pretrained_model} --video ../myCutvideo.mp4 --sf {SLOW_MOTION_FACTOR} --fps {TARGET_FPS} --output ../myoutput.mkv && ffmpeg -i ../myoutput.mkv ../myoutput.mp4

from IPython.display import clear_output


#find the frame_rate of source video
from fractions import Fraction
frame_rate_source = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate myCutvideo.mp4
frame_rate_source = float(Fraction(frame_rate_source[0]))

#if fps > 30 overwrite frame_rate_source = 30
if frame_rate_source > 30:
  frame_rate_source = 30


#slow down both video
!ffmpeg -r {frame_rate_source/SLOW_MOTION_FACTOR} -i myCutvideo.mp4 -y myvideo_slow.mp4
!ffmpeg -r {frame_rate_source} -i myoutput.mp4 -y myoutput_slow.mp4

#combile two videos
!ffmpeg \
  -i myoutput_slow.mp4 \
  -i myvideo_slow.mp4 \
  -filter_complex '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]' \
  -map [vid] \
  -c:v libx264 \
  -crf 23 \
  -preset veryfast \
  -y myoutput_4_display.mp4
clear_output()
show_local_mp4_video('myoutput_4_display.mp4', width=900, height=253, slowmoFactor = 1)




In [ ]:
!nvidia-smi